Automating iterative prewhitening with Pyriod is possible. You have to set it up yourself currently, but I'll eventually add some convenience functions. If there are incoherent signals in the data, this approach will pick up many densely clustered peaks around the main variability frequency, which you can try to recognize later. When it comes to aliasing, it will pick the alias with the highest peak. This approach will not identify combination frequencies (something else I could try to add). Here I imagine we want to prewhiten all frequencies within a specified range of frequencies, above some significance threshold.

In [1]:
%matplotlib widget
import lightkurve as lk
from Pyriod import Pyriod
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Download light curve of a DBV white dwarf pulsator observed by TESS
#Then smooth and remove outliers
lc = lk.search_lightcurve('TIC 257459955',mission='TESS',sector=3)[0].download().remove_outliers(5).flatten(2161)

In [3]:
# We want to prewhiten everything in a range of frequencies.
minfreq = 500 # muHz default for Pyriod
maxfreq = None # Nyquist

# Initialize Pyriod. To make this fast, we do not need it to be handling inteactivity or plots.
pyriod = Pyriod(lc, minfreq=minfreq, maxfreq=maxfreq, gui=False)

# Need a significance threshold to prewhiten down to
# We will update this as we go
pyriod.calculate_significance_threshold(multiplier = 5, # Change scaling factor for significance relative to average noise
                                        startfreq=minfreq,
                                        endfreq=None,
                                        freqstep=100,
                                        winwidth=100,
                                        avgtype='mean')

#What we compare: our significance threshold to the amplitude spectrum of residuals
sigthresh = pyriod.noise_spectrum(pyriod.freqs)*pyriod.significance_multiplier # a bit cumbersome
amplitudespectrum = pyriod.per_resid.power.value # amplitudes
frequencies = pyriod.freqs
abovethreshold = np.where(pyriod.per_resid.power.value > sigthresh)[0]

i=0
while len(abovethreshold) > 0:
    # Identify the frequency with highest significant peak
    peakfreq = frequencies[abovethreshold[np.argmax(amplitudespectrum[abovethreshold])]]
    peakamp = pyriod.interpls(peakfreq) # Amplitude estimate for a good fit
    # I need to address why the values below have to be passes as a list, I thought I have fixed that.
    pyriod.add_signal([peakfreq], amp=[peakamp]) # add highest amplitude peak from periodogram of residuals 
    pyriod.fit_model() # refine least-squares fit to light curve

    # Update significance threshold and current amplitude spectrum of residuals
    pyriod.calculate_significance_threshold(multiplier = 5, 
                                            startfreq=minfreq,
                                            endfreq=None,
                                            freqstep=100,
                                            winwidth=100,
                                            avgtype='mean')
    sigthresh = pyriod.noise_spectrum(pyriod.freqs)*pyriod.significance_multiplier # a bit cumbersome
    amplitudespectrum = pyriod.per_resid.power.value # amplitudes
    abovethreshold = np.where(pyriod.per_resid.power.value > sigthresh)[0]

# This could be a long or infinite loop if you set you significance threshold to something too low
# But hopefully it ends

# Print the frequency solution
print(pyriod.fitvalues)

# Save the frequency solution to be loaded into a Pyriod instance with the gui
pyriod.save_solution(filename = 'autoprewhitened.csv')

     include         freq  fixfreq   freqerr       amp  fixamp    amperr  \
f0      True  1561.199620    False  0.005563  0.025683   False  0.000516   
f1      True  1473.976427    False  0.013710  0.015332   False  0.000824   
f2      True  1673.496956    False  0.009817  0.014554   False  0.000483   
f3      True  3035.236745    False  0.021199  0.006749   False  0.000993   
f4      True  3234.663259    False  0.022745  0.006292   False  0.000506   
f5      True  3122.449479    False  0.029653  0.004826   False  0.000649   
f6      True  1793.245936    False  0.030109  0.004744   False  0.000553   
f7      True  2059.566244    False  0.031747  0.004506   False  0.009835   
f8      True  1653.864412    False  0.034661  0.004124   False  0.000482   
f9      True  1334.576842    False  0.037555  0.003805   False  0.001424   
f10     True  1473.501752    False  0.071454  0.003055   False  0.000737   
f11     True  1154.784949    False  0.047267  0.003021   False  0.000482   

        pha

In [4]:
pyriodgui = Pyriod(lc, minfreq=minfreq, maxfreq=maxfreq, gui=True)
pyriodgui.load_solution(filename = 'autoprewhitened.csv')
# solution is staged, but not fitted to the data yet without a last call to fit_model
pyriodgui.fit_model()
# reculculate and display significance threshold
pyriodgui.calculate_significance_threshold(multiplier = 5, 
                                           startfreq=minfreq,
                                           endfreq=None,
                                           freqstep=100,
                                           winwidth=100,
                                           avgtype='mean')
pyriodgui.Pyriod()

/Users/keatonb/anaconda3/lib/python3.12/site-packages/Pyriod/Pyriod.py:1492: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  + ': {:.8f} '.format(self.fitvalues.freq[i])
